In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/houses.csv")

In [3]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  int64  
 3   total_rooms         20433 non-null  int64  
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  int64  
 6   households          20433 non-null  int64  
 7   median_income       20433 non-null  float64
 8   ocean_proximity     20433 non-null  object 
 9   median_house_value  20433 non-null  int64  
dtypes: float64(4), int64(5), object(1)
memory usage: 1.7+ MB


In [4]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
0,-122.23,37.88,41,880,129.0,322,126,8.3252,NEAR BAY,452600
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,NEAR BAY,358500
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,NEAR BAY,352100
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,NEAR BAY,341300
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,NEAR BAY,342200


Работа с параметрами

In [5]:
def distance(lat1,lon1,lat2,lon2):
    import math
    # Earth Radius in KM 
    R = 6371
    dLat = math.radians(lat2-lat1);
    dLon = math.radians(lon2-lon1);
    lat1 = math.radians(lat1);
    lat2 = math.radians(lat2);
    a = math.sin(dLat/2) * math.sin(dLat/2) +  math.sin(dLon/2) * math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

In [6]:

los_angeles = (34.05224006522778, -118.24340295060638)
san_francisco = (37.768869788439, -122.41781584327025)

In [7]:
df['los_angeles_dist'] = df.apply(lambda x: distance(x.latitude, x.longitude, los_angeles[0], los_angeles[1]), axis=1)
df['san_francisco_dist'] = df.apply(lambda x: distance(x.latitude, x.longitude, san_francisco[0], san_francisco[1]), axis=1)

In [8]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value,los_angeles_dist,san_francisco_dist
0,-122.23,37.88,41,880,129.0,322,126,8.3252,NEAR BAY,452600,556.544948,20.611281
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,NEAR BAY,358500,554.295669,20.115667
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,NEAR BAY,352100,554.626612,18.038918
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,NEAR BAY,341300,555.210185,17.283751
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,NEAR BAY,342200,555.210185,17.283751


In [9]:
ocean = pd.get_dummies(df['ocean_proximity']).astype(int)
ocean.head(3)

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0


In [10]:
df = pd.concat((df, ocean), axis=1)

In [11]:
df.drop(columns=["longitude", "latitude", "ocean_proximity"], inplace=True)

In [12]:
df.head(5)

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,los_angeles_dist,san_francisco_dist,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,41,880,129.0,322,126,8.3252,452600,556.544948,20.611281,0,0,0,1,0
1,21,7099,1106.0,2401,1138,8.3014,358500,554.295669,20.115667,0,0,0,1,0
2,52,1467,190.0,496,177,7.2574,352100,554.626612,18.038918,0,0,0,1,0
3,52,1274,235.0,558,219,5.6431,341300,555.210185,17.283751,0,0,0,1,0
4,52,1627,280.0,565,259,3.8462,342200,555.210185,17.283751,0,0,0,1,0


In [13]:
train_df = df.sample(frac=0.8,random_state=200)
test_df = df.drop(train_df.index)

Обучаем модельку

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [15]:
X = train_df.drop(columns=["median_house_value"])
y = train_df.median_house_value

X_test = train_df.drop(columns=["median_house_value"])
y_test = train_df.median_house_value

In [16]:
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

In [17]:
reg = LinearRegression().fit(x, y)

In [18]:
reg.score(x, y)

0.6498506344865067

In [19]:
reg.coef_

array([ 9.21301409e+02, -4.90475377e+00,  8.25945568e+01, -3.84905411e+01,
        6.37045476e+01,  3.86908359e+04, -1.15028677e+02, -1.17417577e+02,
       -3.46062373e+04, -9.79056567e+04,  1.76224795e+05, -3.54587793e+04,
       -8.25412171e+03])

Посчитаем метрики

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
y_pred = reg.predict(X_test)

/home/blinovma/Projects/ml_experiments/ml-system-design/venv/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [24]:
mean_squared_error(y_test, y_pred, squared=False)

68562.94361324406